<a href="https://www.kaggle.com/code/mcpenguin/red-wine-quality-prediction?scriptVersionId=143235380" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Red Wine Classification

In this notebook, we investigate the red wine dataset and build a model to
predict their relative quality.

# Import Libraries

In [ ]:
# basic libraries / data processing
import numpy as np
import pandas as pd
import os

# graphical visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# modelling
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Load Datasets

In [ ]:
df = pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
df.head()

Let's try to get initial information about this dataset by calling `info` and `describe`:

In [ ]:
df.info()

In [ ]:
df.describe()

From these, we see that there are no missing values.

# Exploratory Data Analysis

## Histograms of Variates

To better understand the distributions of the variates in the data, we can plot histograms for the individual variates.

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(16, 9))
plt.subplots_adjust(hspace=1)

for ax, col in zip(axes.flat, df.columns):
    sns.histplot(data=df, x=col, ax=ax)
    ax.set_title(f"Distribution of {col}")

plt.show()

We see that wine quality is actually a categorical (integer) value. As such, it makes sense to treat it as a category during our EDA.

## Boxenplots of Variates vs. Quality

As the quality of the wine is the thing that we want to predict, a natural graphical visualization we could look at would be distributions of the variates after taking the quality of the wine into account.

In [ ]:
# convert quality to categorical variate
df["quality category"] = df["quality"].astype("category")

fig, axes = plt.subplots(nrows=11, ncols=1, figsize=(16, 100))
# plt.subplots_adjust(hspace=1)
for ax, col in zip(axes.flat, df.columns[:-1]):
    sns.boxenplot(data=df, y=col, x="quality", ax=ax)
    ax.set_title(f"Distribution of {col} vs. Quality")

plt.show()

## Correlation Matrix

We can better understand the correlations between the variates by using a correlation matrix.

In [ ]:
ax = sns.heatmap(df.corr(numeric_only=True))
plt.show()

From this, we see that the variates are relatively uncorrelated. The only contenders of multicollinearity seem to be 

* `fixed acidity` and `citric acid`;

* `free sulfur dioxide` and `total sulfur dioxide`.

We might want to consider removing these variates before modelling to improve model performance.

# Feature Engineering

## Sulfur Dioxide Ratio

Given the trends that we observed in our EDA, a natural variate we could use in place of the sulfur dioxide variates would be the ratio of free vs. total sulfur dioxide in the wine.

In [ ]:
df["sulfur dioxide ratio"] = df["free sulfur dioxide"] / df["total sulfur dioxide"]

We can see the distribution of this transformed variate in our data:

In [ ]:
ax = sns.histplot(data=df, x="sulfur dioxide ratio")
ax.set_title(f"Distribution of sulfur dioxide ratio")
plt.show()

Additionally, we can see the distributions of sulfur dioxide after taking into account the quality of the wine:

In [ ]:
plt.figure(figsize=(12,8))

ax = sns.boxenplot(data=df, y="sulfur dioxide ratio", x="quality category")
plt.show()

# Train/Test Split

Prior to modelling, we will split our data into training, validation and test sets.

In [ ]:
explanatory_cols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'sulfur dioxide ratio', 'density', 'pH', 'sulphates', 'alcohol']
response_col = ['quality']

X = df[explanatory_cols]
y = df[response_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_test: {y_test.shape}")

# Modelling

For our predictive model, we will be trying different regression algorithms and choosing the best one to tune. To do this, we will be employing (stratified) K-fold cross-validation.

In [ ]:
models = [
    ('random_forest', RandomForestRegressor()),
    ('gradient_boosting', GradientBoostingRegressor()),
    ('hist_gradient_boosting', HistGradientBoostingRegressor()),
    ('svr', SVR()),
    ('k_neighbors', KNeighborsRegressor())
]

# Training

In [ ]:
def create_pipeline(name, model):
    return Pipeline([
        ('scaler', StandardScaler()),
        (name, model)
    ])

n_splits = 5
skf = StratifiedKFold(n_splits = n_splits, shuffle=True)

best_loss = np.inf
best_model_name = None

for name, model in models:
    print("-"*25)
    print(f"Training MODEL {name}")
    print("-"*25)
    val_losses = []
    
    for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
        print(f"Training on FOLD {i+1}")
        X_train_act = X_train.iloc[train_index].values
        X_val = X_train.iloc[val_index].values
        y_train_act = y_train.iloc[train_index].values.ravel()
        y_val = y_train.iloc[val_index].values.ravel()
        
        pipe = create_pipeline(name, model)
        
        pipe.fit(X_train_act, y_train_act)
        y_val_pred = pipe.predict(X_val)
        mse = mean_squared_error(y_val, y_val_pred)
        print(f"MSE on validation set: {mse}")
        print()
        
        val_losses.append(mse)
    
    average_loss = np.mean(val_losses)
    print(f"Average MSE: {average_loss}")
    if average_loss < best_loss:
        best_loss = average_loss
        best_model_name = name

print()
print(f"Best model: {best_model_name}")
print(f"MSE: {best_loss}")

# Hyperparameter Tuning

We can further tune the parameters of the `RandomForest` model using `GridSearchCV` to try to get a better score.

In [ ]:
models_dict = dict(models)
best_model = models_dict[best_model_name]

pipeline = create_pipeline(best_model_name, best_model)
parameters_grid = {
    f"random_forest__n_estimators": [30, 50, 100, 200],
    f"random_forest__max_depth": [None, 4, 6],
    f"random_forest__max_features": [None, "sqrt", "log2"]
}
search = GridSearchCV(pipeline, parameters_grid)

search.fit(X_train, y_train.values.ravel())

We can see the best parameters:

In [ ]:
search.best_params_

# Feature Importance

We can next investigate the relative feature importance of our model.

In [ ]:
best_pipeline = search.best_estimator_
feature_importances = best_pipeline.named_steps["random_forest"].feature_importances_

plt.figure(figsize=(12, 8))
plt.xticks(rotation = 60)

ax = sns.barplot(x=explanatory_cols, y=feature_importances)
plt.show()

# Prediction

We can now predict the wine quality scores on the test dataset.

In [ ]:
y_pred = best_pipeline.predict(X_test)
print(f"MSE for test dataset: {mean_squared_error(y_pred, y_test)}")

To see the accuracy of the predictions, we can plot the predictions and the actual ratings for the test dataset on a scatterplot. If our predictive model is good, we should expect to see that the points should be close to the line `y = x`.

In [ ]:
flat_y_test = y_test.values.flatten()
ax = sns.scatterplot(x=y_pred, y=flat_y_test)
sns.lineplot(x=flat_y_test, y=flat_y_test, ax=ax)
ax.set_xlabel("Predicted Rating")
ax.set_ylabel("Actual Rating")
plt.show()

We see that our model does a pretty good job at predicting the wine rating.

# Thanks for reading!